<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import pickle

def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

In [5]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [6]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [7]:
def calc_results(model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbmgs.best_score_ 
    result_dict['best_index'] = lgbmgs.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
    return result_dict

In [8]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [9]:
com_name = 'sec'
fname = f'df_{com_name}_sel.pkl'
directory_for_ml = '../data/data_for_ml/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [10]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [11]:
# params_test= { 
#     "num_iterations": [100, 200, 300, 400, 500],
#     "num_class": [1]
#     "metric" : "rmse",
#     "bagging_frequency" : 5,
#     "bagging_seed" : 2018,
#     "verbosity" : -1,

#     # Selected rounded-off params
#     'bagging_fraction': 0.7,
#     'feature_fraction': 0.1,
#     'lambda_l1': 1,
#     'lambda_l2': 0,
#     'max_depth': 9,
#     'min_child_weight': 5,
#     'min_split_gain': 0,
#     'num_leaves': 24 }

In [12]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [300], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.03, 0.04, 0.051, 0.06, 0.07], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.4, 0.5, 0.6, 0.7, 0.8], # best value found, default = 0
    "scale_pos_weight": [2, 10, 15], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
#     "is_unbalance": [True],  # 불균형 셋 조정. 주의: "scale_pos_weight"과 동시에 사용할 수 없음
    #---------------------------------------------
#     "scoring": ['accuracy', 'precision'], 
#     "early_stopping_rounds" : [200],
  
# ************* 아래 하나씩 테스트해서 취사 선택해야 함. **************
#     "bagging_frequency" : [5], 
#     "num_threads": [0], # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "min_data_in_leaf": [None], # 과적합 방지 파라미터
#     "lambda_l1": [0], # default 0
#     "lambda_l2": [0], #default 0
#     "min_gain_to_split": [None], # 분기하기 위해 필요한 최소한의 gain을 의미, 정규화시 사용
#     "num_threads": = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "max_bin": [32],  # data 준비할 때 사용하는 parameter     

#         parameters = {'learning_rate': [0.01,0.02,0.03],
#                   'subsample'    : [0.9, 0.5, 0.2],
#                   'n_estimators' : [100,500,1000],
#                   'max_depth'    : [4,6,8]
#                  }
    
}

In [13]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
#     "num_iterations" : [200], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.06, 0.065, 0.07, 0.075, 0.08], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [80, 100, 120],
    "max_delta_step": [ 0.8, 0.9], # best value found, default = 0
    "scale_pos_weight": [10], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
#     "scoring": ['accuracy', 'precision'],   

}

In [14]:
def new_num_iter(num_i):
    if num_i <= 10:
        new_num = [10, 50, 100]
    elif num_i <= 100:
        new_num = [50, 100, 150, 200]
    elif num_i <= 200:
        new_num = [150, 200, 300]
    else:
        new_num = [num_i - 100, num_i, num_i + 100]
    return new_num

def new_lr(lr):
    
    def count_zero(x):
        if x < 1:
            iter = 1
            while True:
                if x * 10**iter >= 1:
                    break
                iter = iter+1
            return iter
        elif x >= 1:
            iter = 1
            while True:
                if x / 10**iter < 1:
                    break
                iter = iter+1
            return -iter
    
    def back_step(lr, lr_step):
        if lr - lr_step*2 <=0 :
            tenth = 10**(count_zero(lr)+1)
            return int(lr*tenth/3)/tenth, int(lr*tenth/2)/tenth
        else:
            return lr_step, lr_step*2
    
    if lr <= 0.001:
        lr_step = 0.0002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.01:
        lr_step = 0.002
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 0.1:
        lr_step = 0.02
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 1:
        lr_step = 0.2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    elif lr <= 10:
        lr_step = 2
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)
    else:
        lr_step = 20
        lr_step_m1, lr_step_m2 = back_step(lr, lr_step)

    return [lr-lr_step_m2, lr-lr_step_m1, lr, lr+lr_step, lr+lr_step*2]

def new_depth(depth):
    if depth == None:
        new_depth = [None, 1, 2, 3, 4]
        new_depth = [1, 2, 3, 4]
    elif depth <= 2:
        new_depth = [1, 2, 3, 4]
    elif depth <= 4:
        new_depth = [3, 4, 5, 6]
    elif depth <= 6:
        new_depth = [5, 6, 7, 8]
    else:
        new_depth = [7, 8, 9, 10, 11]
    return new_depth

def new_leaves(leaves):
    if leaves <= 3:
        new_leaves = [2, 3, 4]
    elif leaves <= 10:
        new_leaves = [leaves-2, leaves-1, leaves, leaves+1, leaves+2]
    elif leaves <= 20:
        new_leaves = [leaves-4, leaves-2, leaves, leaves+2, leaves+4]
    else:
        new_leaves = [leaves-8, leaves-4, leaves, leaves+4, leaves+8]
    return new_leaves

def new_subsample(subsam):
    if subsam == 0.002:
        return [0.0005, 0.001, 0.002, 0.003]
    elif subsam <= 0.01:
        step = 0.001
    elif subsam <= 0.1:
        step = 0.01
    elif subsam <= 1:
        step = 0.2
        
    return [subsam-step, subsam, subsam+step]
    
def new_nestimator(nest):
    if nest <= 10:
        return [5, 10, 30]
    elif nest <= 50:
        return [10, 30, 50, 70]
    elif nest <= 100:
        return [60, 80, 100, 120]
    elif nest <= 140:
        return [100, 120, 140, 160]
    else :
        return [140, 160, 190, 220]
    
def make_new_parameter(params_o, df_concat):
    param_new = {}
    param_new["boosting_type"] = ['gbdt']
    
    num_i = df_concat.loc["num_iterations"].iloc[0]
    param_new["num_iterations"] = new_num_iter(num_i) #새로운 값을 중간에 배치, 100단위로 순서
    
    lr = df_concat.loc["learning_rate"].iloc[0]
    param_new["learning_rate"] = new_lr(lr)
    
    depth = df_concat.loc["max_depth"].iloc[0]
    param_new["max_depth"] = new_depth(depth) # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    
    leaves = df_concat.loc["num_leaves"].iloc[0]  
    param_new["num_leaves"] = new_leaves(leaves) #  # 두번째로 중요, num_leaves = 2^max_depth보다 작아야 함. 데이터가 적으면 작은 숫자로
    
    subsam = df_concat.loc["subsample"].iloc[0]  
    param_new["subsample"] = new_subsample(subsam) # 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    
    param_new["colsample_bytree"] = [0.25, 0.3, 0.35, 0.4, 0.45] # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    param_new["objective"] = ['binary']
    param_new["metric"] = ['binary_logloss'] # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    nest = df_concat.loc["n_estimators"].iloc[0] 
    param_new["n_estimators"] = new_nestimator(nest) # [None, 3, 4, 5]
    param_new["max_delta_step"] = [0.5, 0.6, 0.7, 0.8, 0.9] # best value found, default = 0
    param_new["scale_pos_weight"] = [3, 10, 20] #  class imbalance 경감, scale_pos_weight > 0.0, defaut
#     param_new["is_unbalance"] = [1],  # 불균형 셋 조정, 주의: "scale_pos_weight" 동시에 사용할 수 없음.
    
#     param_new["scoring"] = ['accuracy', 'precision'],
#     param_new["early_stopping_rounds"] = [50],

    
    print("-------- End of Parameter Setting -------")
    
    return param_new

In [15]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)
    
iter = 0
while True:
    iter = iter + 1

    lgbm = None
    lgbmgs = None

    lgbm = lightgbm.LGBMClassifier(random_state=42)

    lgbmgs = GridSearchCV(estimator = lgbm,
                          param_grid = params_o,
                          cv = 5, # StratifiedKFold us default for binary or multiclass
                          scoring = 'precision', 
#                           scoring = 'accuracy', 
#                           scoring = ['accuracy', 'precision'], # refit 사용해야 함. 고로 사용하지 않음.
                          error_score='raise',
                          verbose = 1,
                          n_jobs=-1, # 자동 검색 적용
                         )
                          
    print("*** after lgbmgs ******")
    lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') 
#     lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss', eval_set = (val_scaled, val_target)) 
    # eval_set가 있어야 "early_stopping_rounds"를 사용할 수 있음.

# save model
    stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
    dt = re.sub(r'[-:T]', '', stamp[5:16])
    joblib.dump(lgbmgs, f'./{com_name}/lgbm_v{iter}_{dt}.pkl')
    
    df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmgs.best_estimator_, 
                               train_scaled, val_scaled, test_scaled,  
                               train_target, val_target, test_target)
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_concat = pd.concat([df_estimator, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        df_base.to_csv(f'./{com_name}/lgbm_ver{iter}_df_{dt}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_ver{iter}_df_{dt}.pkl')
        break
    if iter >= 7 : 
        df_base.to_csv(f'./{com_name}/lgbm_ver{iter}_df_{dt}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_ver{iter}_df_{dt}.pkl')
        break
    print("******* No.{}  Process is Done! ********".format(iter))
    params_o = make_new_parameter(params_o, df_concat)
    save_to_pickle(f'./{com_name}/params_ver{iter}_{dt}.pkl', params_o)
    
print("**** End of Process ****")
# save model, save df, stoppping 기준 수립
# 일단위로 정확도 측정, 정확도, 정밀도?

# eval_set, early_stopping_rounds 내용 확인 필. ">" 에러 확인.
# 집어 넣으면 에러 발생하니, 원인을 찾아야 함. 하면 좋은 이유는?

# scoring = ['accuracy', 'precision'] 두가지 param 선언이 안되나?
# Precision is ill-defined and being set to 0.0 due to no predicted samples.  왜 나오나 확인해야 함.
# .predict로 하면 positive로 예측하는 경우가 발생하지 않음. 


# feature_importance 확인하고 중요한 feature만 갖고 분석해보기

*** after lgbmgs ******
Fitting 5 folds for each of 12000 candidates, totalling 60000 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


******* No.1  Process is Done! ********
-------- End of Parameter Setting -------
*** after lgbmgs ******
Fitting 5 folds for each of 202500 candidates, totalling 1012500 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


******* No.2  Process is Done! ********
-------- End of Parameter Setting -------
*** after lgbmgs ******
Fitting 5 folds for each of 337500 candidates, totalling 1687500 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


******* No.3  Process is Done! ********
-------- End of Parameter Setting -------
*** after lgbmgs ******
Fitting 5 folds for each of 337500 candidates, totalling 1687500 fits


C:\Users\kange\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


******* No.4  Process is Done! ********
-------- End of Parameter Setting -------
*** after lgbmgs ******
Fitting 5 folds for each of 337500 candidates, totalling 1687500 fits


KeyboardInterrupt: 

In [16]:
df_base.to_csv(f'./{com_name}/lgbm_ver{iter}_df_{dt}.csv')
df_base.to_pickle(f'./{com_name}/lgbm_ver{iter}_df_{dt}.pkl')

In [ ]:
model = lgbmgs.best_estimator_ 

In [ ]:
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [ ]:
feature_df

In [ ]:
df_base

In [ ]:
df_base.to_csv(f'./{com_name}_lgbm_ver000_df.csv')
df_base.to_pickle(f'./{com_name}_lgbm_ver000_df_______ppp.pkl')

In [ ]:
df_base.to_pickle(f'./{com_name}_lgbm_ver000_df_______pppXXXXX.pkl')

In [ ]:
joblib.dump(lgbmgs, f'./{com_name}_lgbm_vVV.pkl') 

In [ ]:
com_name

In [ ]:
joblib.dump(lgbmgs, f'./{com_name}/lgbm_v{iter}_{stamp[5:16]}.pkl')

In [ ]:
dt = re.sub(r'[-:T]', '', stamp[5:16])
joblib.dump(lgbmgs, f'./{com_name}/lgbm_v{iter}_{dt}.pkl')

In [ ]:
stamp = datetime.datetime.today().isoformat()
stamp[5:16]

In [ ]:
stamp[5:16].replace(["-","T",":"],"O")

In [ ]:
backs = re.sub(r'[\\.]', '*', 'sam.sun\\g') 
backs

In [ ]:

backs = re.sub(r'[-:T]', '', stamp[5:16]) 
backs

In [ ]:
import re

In [ ]:
stamp = datetime.datetime.today().isoformat()[5:16] # 파일명 끝에 생성날짜 시간 추가
joblib.dump(lgbmgs, f'./{com_name}_lgbm_ver{iter}_{stamp}_____1.pkl') 

In [ ]:
joblib.dump(lgbmgs, f'./{com_name}_lgbm_ver{iter}_{stamp}OOOOOOOOPP.pkl') 

In [ ]:
# save the results
directory = './'
df_base.to_pickle(directory + f'df_{com_name}_lgbm.pkl')
df_base.to_csv(directory + f'df_{com_name}_lgbm.csv')

In [ ]:
## end

In [ ]:
df_base

In [ ]:
df_temp = pd.read_csv(f'./{com_name}_lgbm_ver{iter}_df.csv')

In [ ]:
df_temp

In [ ]:
params_o = make_new_parameter(params_o, df_concat)

In [ ]:
iter =0 
iter = iter + 1

lgbm = None
lgbmgs = None

lgbm = lightgbm.LGBMClassifier( random_state=42)

lgbmgs = GridSearchCV(estimator = lgbm,
                      param_grid = params_o,
                      cv = 5, # StratifiedKFold us default for binary or multiclass
#                       scoring = 'precision', 
                      scoring = 'accuracy', 
                      error_score='raise',
                      verbose = 1,
                      n_jobs=-1, # 자동 검색 적용
                      )

lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') # 

df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
df_result = make_df_from_estimator(result_dict, iter)
df_concat = pd.concat([df_estimator, df_result])

df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

In [ ]:
print("Best Estimator: {}".format(lgbmgs.best_estimator_)) 
print("Best Parameters: {}".format(lgbmgs.best_params_))  # 최적 파라미터.
print('Best Score: {}, Best Index: {}'.format(lgbmgs.best_score_ , lgbmgs.best_index_))  # 교차검증된 점수를 보여줌.

model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_pred = model.predict(test_scaled)

train_score = model.score(train_scaled, train_target)
val_score = model.score(val_scaled, val_target)
test_score = model.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, model.predict(test_scaled))
cm = confusion_matrix(test_target, model.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score, val_score, test_score))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

In [ ]:
iter=5
a = f'./{com_name}_lgbm_ver{iter}.h5'
a

In [ ]:
model.feature_importances_

In [ ]:
df_sel = pd.DataFrame(model.feature_importances_, index=data.columns, columns=['importance']).sort_values(by='importance', ascending=False)

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

In [ ]:
# # 최초의 empty df 생성
# df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [ ]:
df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), 1)
result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
df_result = make_df_from_estimator(result_dict, 1)
df_concat = pd.concat([df_estimator, df_result])

In [ ]:
df_concat

In [ ]:
df_concat.index =['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type',
       'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight',
       'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective',
       'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample',
       'subsample_for_bin', 'subsample_freq', 'max_delta_step', 'metric',
       'num_iterations', 'scale_pos_weight', 'best_score', 'best_index',
       'acc_train', 'acc_val', 'acc_test', 'precision', 'recall', 'f1score',
       'roc', 'tn', 'fp', 'fn', 'tp']
df_concat

In [ ]:
df_base

In [ ]:
df_base

In [ ]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [ ]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [ ]:
# dfs = [df_estimator, df_result]
# df_merged= reduce(lambda  left,right: pd.merge(left,right, how='outer', left_index=True, right_index=True), dfs)

In [ ]:
df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

In [ ]:
pa = params_o

In [ ]:
params_o["boosting_type"] == pa["boosting_type"]

In [ ]:
param_new["max_delta_step"] = [0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
params_o["max_delta_step"]

In [ ]:
params_o["num_iterations"] = new_num_iter(50)

In [ ]:
params_o["num_iterations"]

In [ ]:
pp = make_new_parameter(params_o, df_concat)

In [ ]:
pp

In [ ]:
new_num_iter(20)

In [ ]:
aaa = {}
aaa['bb'] = new_num_iter(20)
aaa

In [ ]:
aaa['bb']

In [ ]:
aaa["boosting_type"] = ['gbdt']

In [ ]:
model = lgbmgs.best_estimator_
model.save(f'./{com_name}_lgbm_ver{iter}.h5')

In [ ]:
joblib.dump(lgbmgs, f'./{com_name}_lgbm_ver{iter}.pkl') 

In [ ]:
joblib.dump(lgbmgs, 'testmodel.pkl') 

In [ ]:
loadmodel = joblib.load('testmodel.pkl') 

In [ ]:
model = loadmodel.best_estimator_

In [ ]:
model.best_score_

In [ ]:
params_o

In [ ]:
directory = './'
df_temp = pd.read_pickle(directory + f'df_{com_name}_lgbm.pkl')

In [ ]:
df_concat

In [ ]:
def new_num_iter(num_i):
    if num_i <= 10:
        new_num = [10, 50, 100]
    elif num_i <= 100:
        new_num = [50, 100, 150, 200]
    elif num_i <= 200:
        new_num = [150, 200, 300]
    else:
        new_num = [num_i - 100, num_i, num_i + 100]
    return new_num

for k in range(1, 300, 5):
    a = new_num_iter(k)
    print("kk", k, "list: ", a)

In [ ]:
stamp = datetime.datetime.today().isoformat()[5:16] # 파일명 끝에 생성날짜 시간 추가
joblib.dump(lgbmgs, f'./{com_name}_lgbm_1111.pkl') 

In [ ]:
lgbmgs.best_estimator_.booster_.feature_importance(importance_type='gain')

In [ ]:
aa = joblib.load('mobis_lgbm_1111.pkl')

In [ ]:
aa

In [ ]:
import os
com_name = 'sec'
if not os.path.exists(com_name):
    os.makedirs(com_name)

In [ ]:
max_depth = [1,2,3,4,5],
njm_leaves = [3,4,5,6,7],
learning_rate = [0.03, 0.05, 0.1, 0.2, 0.3]
colsample_bytree = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
subsample = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
n_estimator =[5, 10, 15, 20]
num_class = [1]

num_leaves = 20~3000
max_depth = 3 ~12
learning_rate = 0.01 ~ 0.3

parameters = {'learning_rate': [0.01,0.02,0.03],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4,6,8]

In [ ]:
sec
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
train accuracy: 1.0000, val accuracy 0.7907, test accuracy 0.8519
precision : 0.7368, recall : 0.8235, f1score : 0.7778, roc : 0.8442
[[32  5]
 [ 3 14]]

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.41, eval_metric='logloss',
               learning_rate=0.004, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=5, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.005)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.41, 'learning_rate': 0.004, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.005}
Best Score: 0.9333333333333332, Best Index: 0
train accuracy: 0.8081, val accuracy 0.7442, test accuracy 0.7593
precision : 1.0000, recall : 0.2353, f1score : 0.3810, roc : 0.6176
[[37  0]
 [13  4]]

Fitting 5 folds for each of 18000 candidates, totalling 90000 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.5 will be ignored. Current value: bagging_fraction=0.9
Best Estimator: LGBMClassifier(bagging_fraction=0.9, colsample_bytree=0.5, feature_fraction=0.8,
               learning_rate=0.05, max_delta_step=0.5, max_depth=2,
               metric='auc', num_iterations=500, num_leaves=4,
               objective='binary', random_state=42, subsample=0.5)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'metric': 'auc', 'n_estimators': 100, 'num_leaves': 4, 'objective': 'binary', 'subsample': 0.5}
Best Score: 0.8375180375180376, Best Index: 726
train accuracy: 1.0000, val accuracy 0.7907, test accuracy 0.8148
precision : 0.7059, recall : 0.7059, f1score : 0.7059, roc : 0.7854
[[32  5]
 [ 5 12]]

Fitting 5 folds for each of 7200 candidates, totalling 36000 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.3 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.3 will be ignored. Current value: bagging_fraction=0.9
Best Estimator: LGBMClassifier(bagging_fraction=0.9, colsample_bytree=0.3, feature_fraction=0.8,
               max_delta_step=0.5, max_depth=4, metric='rmse', n_estimators=80,
               num_iterations=500, num_leaves=6, objective='binary',
               random_state=42, subsample=0.3)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'metric': 'rmse', 'n_estimators': 80, 'num_leaves': 6, 'objective': 'binary', 'subsample': 0.3}
Best Score: 0.8377777777777778, Best Index: 915
train accuracy: 1.0000, val accuracy 0.7907, test accuracy 0.8519
precision : 0.7368, recall : 0.8235, f1score : 0.7778, roc : 0.8442
[[32  5]
 [ 3 14]]

Fitting 5 folds for each of 12500 candidates, totalling 62500 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.1 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.1 will be ignored. Current value: bagging_fraction=0.9
Best Estimator: LGBMClassifier(bagging_fraction=0.9, colsample_bytree=0.1, feature_fraction=0.8,
               learning_rate=0.07, max_delta_step=0.5, max_depth=5,
               metric='rmse', n_estimators=50, num_iterations=500, num_leaves=6,
               objective='binary', random_state=42, subsample=0.1)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.07, 'max_depth': 5, 'metric': 'rmse', 'n_estimators': 50, 'num_leaves': 6, 'objective': 'binary', 'subsample': 0.1}
Best Score: 0.8377777777777778, Best Index: 315
train accuracy: 1.0000, val accuracy 0.8140, test accuracy 0.8519
precision : 0.7368, recall : 0.8235, f1score : 0.7778, roc : 0.8442
[[32  5]
 [ 3 14]]

Fitting 5 folds for each of 12500 candidates, totalling 62500 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.4, learning_rate=0.03, max_delta_step=0.5,
               max_depth=3, metric='loss', n_estimators=30, num_iterations=500,
               num_leaves=4, objective='binary', random_state=42,
               subsample=0.1)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.03, 'max_depth': 3, 'metric': 'loss', 'n_estimators': 30, 'num_leaves': 4, 'objective': 'binary', 'subsample': 0.1}
Best Score: 0.8342857142857143, Best Index: 7605
train accuracy: 1.0000, val accuracy 0.7907, test accuracy 0.8704
precision : 0.7500, recall : 0.8824, f1score : 0.8108, roc : 0.8736
[[32  5]
 [ 2 15]]

Fitting 5 folds for each of 13500 candidates, totalling 67500 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.35, eval_metric='logloss', learning_rate=0.02,
               max_delta_step=0.5, max_depth=3, metric='loss', n_estimators=10,
               num_iterations=100, num_leaves=4, objective='binary',
               random_state=42, subsample=0.05)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.35, 'learning_rate': 0.02, 'max_depth': 3, 'metric': 'loss', 'n_estimators': 10, 'num_iterations': 100, 'num_leaves': 4, 'objective': 'binary', 'subsample': 0.05}
Best Score: 1.0, Best Index: 1204
train accuracy: 0.7849, val accuracy 0.7442, test accuracy 0.7037
precision : 1.0000, recall : 0.0588, f1score : 0.1111, roc : 0.5294
[[37  0]
 [16  1]]

Fitting 5 folds for each of 2700 candidates, totalling 13500 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.45, eval_metric='logloss', learning_rate=0.01,
               max_delta_step=0.5, max_depth=2, metric='loss', n_estimators=10,
               num_iterations=300, num_leaves=3, objective='binary',
               random_state=42, subsample=0.05)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.45, 'learning_rate': 0.01, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 10, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.05}
Best Score: 0.8638888888888889, Best Index: 1800
train accuracy: 0.9128, val accuracy 0.8140, test accuracy 0.8519
precision : 0.8000, recall : 0.7059, f1score : 0.7500, roc : 0.8124
[[34  3]
 [ 5 12]]

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.4, eval_metric='logloss', learning_rate=0.01,
               max_delta_step=0.5, max_depth=2, metric='loss', n_estimators=5,
               num_iterations=200, num_leaves=3, objective='binary',
               random_state=42, subsample=0.03)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.01, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.03}
Best Score: 0.9333333333333332, Best Index: 144
train accuracy: 0.8023, val accuracy 0.7442, test accuracy 0.7593
precision : 1.0000, recall : 0.2353, f1score : 0.3810, roc : 0.6176
[[37  0]
 [13  4]]

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.42, eval_metric='logloss', learning_rate=0.01,
               max_delta_step=0.5, max_depth=2, metric='loss', n_estimators=5,
               num_iterations=300, num_leaves=3, objective='binary',
               random_state=42, subsample=0.03)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.42, 'learning_rate': 0.01, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.03}
Best Score: 0.8638888888888889, Best Index: 720
train accuracy: 0.9070, val accuracy 0.8140, test accuracy 0.8519
precision : 0.8000, recall : 0.7059, f1score : 0.7500, roc : 0.8124
[[34  3]
 [ 5 12]]

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.42, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=5, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.01)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.42, 'learning_rate': 0.005, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.01}
Best Score: 0.8355555555555556, Best Index: 432
train accuracy: 0.8895, val accuracy 0.8372, test accuracy 0.8519
precision : 0.8462, recall : 0.6471, f1score : 0.7333, roc : 0.7965
[[35  2]
 [ 6 11]]

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.41, eval_metric='logloss',
               learning_rate=0.004, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=5, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.005)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.41, 'learning_rate': 0.004, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.005}
Best Score: 0.9333333333333332, Best Index: 0
train accuracy: 0.8081, val accuracy 0.7442, test accuracy 0.7593
precision : 1.0000, recall : 0.2353, f1score : 0.3810, roc : 0.6176
[[37  0]
 [13  4]]



In [ ]:
naver

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
train accuracy: 1.0000, val accuracy 0.8205, test accuracy 0.7083
precision : 0.6111, recall : 0.6111, f1score : 0.6111, roc : 0.6889
[[23  7]
 [ 7 11]]

Fitting 5 folds for each of 4608 candidates, totalling 23040 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 864
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.41, eval_metric='logloss',
               learning_rate=0.004, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=5, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.005)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.41, 'learning_rate': 0.004, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.005}
Best Score: 0.9444444444444444, Best Index: 0
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7917
precision : 0.9000, recall : 0.5000, f1score : 0.6429, roc : 0.7333
[[29  1]
 [ 9  9]]

Fitting 5 folds for each of 3456 candidates, totalling 17280 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.003, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.003, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.9777777777777779, Best Index: 216
train accuracy: 0.8170, val accuracy 0.8462, test accuracy 0.7292
precision : 0.8571, recall : 0.3333, f1score : 0.4800, roc : 0.6500
[[29  1]
 [12  6]]

#### scoring을 precision에서 default(accuracy)로 변경한 결과
Fitting 5 folds for each of 3456 candidates, totalling 17280 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 648
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]

#### max_depth = None 삽입결과

Fitting 5 folds for each of 4608 candidates, totalling 23040 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 864
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]

In [ ]:
skhinix

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.36, learning_rate=0.008, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=500, num_leaves=3, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.008, 'max_depth': None, 'metric': 'loss', 'n_estimators': None, 'num_leaves': 3, 'objective': 'binary', 'subsample': None}
Best Score: 0.8201426024955436, Best Index: 1443
train accuracy: 0.8795, val accuracy 0.8571, test accuracy 0.8269
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
[[31  2]
 [ 7 12]]
4 values from sklearn
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
                
                
Fitting 5 folds for each of 4608 candidates, totalling 23040 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.39, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.39, 'learning_rate': 0.005, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.7782531194295901, Best Index: 2016
train accuracy: 0.8434, val accuracy 0.7857, test accuracy 0.8077
precision : 0.8462, recall : 0.5789, f1score : 0.6875, roc : 0.7592
[[31  2]
 [ 8 11]]         

Fitting 5 folds for each of 7200 candidates, totalling 36000 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.36, eval_metric='logloss',
               learning_rate=0.007, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.001)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.007, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.001}
Best Score: 0.8083778966131907, Best Index: 1152
train accuracy: 0.8795, val accuracy 0.8571, test accuracy 0.8269
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
[[31  2]
 [ 7 12]]

Fitting 5 folds for each of 4320 candidates, totalling 21600 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.36, learning_rate=0.011, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=3,
               num_iterations=500, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.011, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.8199643493761141, Best Index: 3024
train accuracy: 0.8675, val accuracy 0.8095, test accuracy 0.8462
precision : 0.8667, recall : 0.6842, f1score : 0.7647, roc : 0.8118
[[31  2]
 [ 6 13]]

Fitting 5 folds for each of 2304 candidates, totalling 11520 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.34, learning_rate=0.01, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=500, num_leaves=3, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.34, 'learning_rate': 0.01, 'max_depth': None, 'metric': 'loss', 'n_estimators': None, 'num_leaves': 3, 'objective': 'binary', 'subsample': None}
Best Score: 0.8199643493761141, Best Index: 3
train accuracy: 0.8916, val accuracy 0.9048, test accuracy 0.8462
precision : 0.8667, recall : 0.6842, f1score : 0.7647, roc : 0.8118
[[31  2]
 [ 6 13]]

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.36, learning_rate=0.008, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=500, num_leaves=3, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.008, 'max_depth': None, 'metric': 'loss', 'n_estimators': None, 'num_leaves': 3, 'objective': 'binary', 'subsample': None}
Best Score: 0.8201426024955436, Best Index: 2307
train accuracy: 0.8795, val accuracy 0.8571, test accuracy 0.8269
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
[[31  2]
 [ 7 12]]

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.36, learning_rate=0.008, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=500, num_leaves=3, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.008, 'max_depth': None, 'metric': 'loss', 'n_estimators': None, 'num_leaves': 3, 'objective': 'binary', 'subsample': None}
Best Score: 0.8201426024955436, Best Index: 1443
train accuracy: 0.8795, val accuracy 0.8571, test accuracy 0.8269
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
[[31  2]
 [ 7 12]]
4 values from sklearn
precision : 0.8571, recall : 0.6316, f1score : 0.7273, roc : 0.7855
                
                

In [ ]:
mobis

Best Estimator: LGBMClassifier(colsample_bytree=0.4, learning_rate=0.02, max_delta_step=0.8,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=450, num_leaves=3, objective='binary',
               random_state=42, scale_pos_weight=2, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.02, 'max_delta_step': 0.8, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 450, 'num_leaves': 3, 'objective': 'binary', 'scale_pos_weight': 2, 'subsample': None}
Best Score: 0.7913043478260869, Best Index: 84963
train accuracy: 0.9826, val accuracy 0.8621, test accuracy 0.7568
precision : 0.6667, recall : 0.7143, f1score : 0.6897, roc : 0.7484
[[18  5]
 [ 4 10]]

Fitting 5 folds for each of 34560 candidates, totalling 172800 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.36, learning_rate=0.006, max_delta_step=0.5,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=500, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.36, 'learning_rate': 0.006, 'max_delta_step': 0.5, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 500, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.8159420289855073, Best Index: 30690
train accuracy: 0.8487, val accuracy 0.7667, test accuracy 0.7895
precision : 0.8182, recall : 0.6000, f1score : 0.6923, roc : 0.7565
[[21  2]
 [ 6  9]]

Fitting 5 folds for each of 34560 candidates, totalling 172800 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.25, learning_rate=0.007, max_delta_step=0.5,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=500, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.25, 'learning_rate': 0.007, 'max_delta_step': 0.5, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 500, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.7652173913043478, Best Index: 4770
train accuracy: 0.8174, val accuracy 0.8621, test accuracy 0.7568
precision : 1.0000, recall : 0.3571, f1score : 0.5263, roc : 0.6786
[[23  0]
 [ 9  5]]

Best Estimator: LGBMClassifier(colsample_bytree=0.25, learning_rate=0.007, max_delta_step=0.5,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=500, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.25, 'learning_rate': 0.007, 'max_delta_step': 0.5, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 500, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.7652173913043478, Best Index: 18585
train accuracy: 0.8174, val accuracy 0.8621, test accuracy 0.7568
precision : 1.0000, recall : 0.3571, f1score : 0.5263, roc : 0.6786
[[23  0]
 [ 9  5]]

Best Estimator: LGBMClassifier(colsample_bytree=0.25, learning_rate=0.007, max_delta_step=0.5,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=500, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.25, 'learning_rate': 0.007, 'max_delta_step': 0.5, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 500, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.7652173913043478, Best Index: 18585
train accuracy: 0.8174, val accuracy 0.8621, test accuracy 0.7568
precision : 1.0000, recall : 0.3571, f1score : 0.5263, roc : 0.6786
[[23  0]
 [ 9  5]]

Best Estimator: LGBMClassifier(colsample_bytree=0.25, learning_rate=0.008, max_delta_step=0.7,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=500, num_leaves=4, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.25, 'learning_rate': 0.008, 'max_delta_step': 0.7, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 500, 'num_leaves': 4, 'objective': 'binary', 'subsample': None}
Best Score: 0.773913043478261, Best Index: 20319
train accuracy: 0.9217, val accuracy 0.8966, test accuracy 0.7027
precision : 0.7143, recall : 0.3571, f1score : 0.4762, roc : 0.6351
[[21  2]
 [ 9  5]]

Best Estimator: LGBMClassifier(colsample_bytree=0.35, learning_rate=0.01, max_delta_step=0.8,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=550, num_leaves=4, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.35, 'learning_rate': 0.01, 'max_delta_step': 0.8, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 550, 'num_leaves': 4, 'objective': 'binary', 'subsample': None}
Best Score: 0.7826086956521741, Best Index: 106596
train accuracy: 0.9826, val accuracy 0.8966, test accuracy 0.7568
precision : 0.7273, recall : 0.5714, f1score : 0.6400, roc : 0.7205
[[20  3]
 [ 6  8]]

Best Estimator: LGBMClassifier(colsample_bytree=0.4, learning_rate=0.02, max_delta_step=0.8,
               max_depth=None, metric='binary_logloss', n_estimators=None,
               num_iterations=450, num_leaves=3, objective='binary',
               random_state=42, scale_pos_weight=2, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.02, 'max_delta_step': 0.8, 'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, 'num_iterations': 450, 'num_leaves': 3, 'objective': 'binary', 'scale_pos_weight': 2, 'subsample': None}
Best Score: 0.7913043478260869, Best Index: 84963
train accuracy: 0.9826, val accuracy 0.8621, test accuracy 0.7568
precision : 0.6667, recall : 0.7143, f1score : 0.6897, roc : 0.7484
[[18  5]
 [ 4 10]]

In [2]:
a = "colsample_bytree=0.4, learning_rate=0.02, max_delta_step=0.8, \
               max_depth=None, metric='binary_logloss', n_estimators=None, \
               num_iterations=450, num_leaves=3, objective='binary', \
               random_state=42, scale_pos_weight=2, subsample=None"

In [13]:
np.array(a.split(", ")).replace(" ", "")

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [12]:
a.split(", ")

['colsample_bytree=0.4',
 'learning_rate=0.02',
 'max_delta_step=0.8',
 '               max_depth=None',
 "metric='binary_logloss'",
 'n_estimators=None',
 '               num_iterations=450',
 'num_leaves=3',
 "objective='binary'",
 '               random_state=42',
 'scale_pos_weight=2',
 'subsample=None']

In [14]:
np.array([1,2]).

array([1, 2])

In [15]:
import pandas as pd

In [54]:
df = pd.DataFrame(a.split(", "), columns=['var'])

In [59]:
df['var1'] = df['var'].str.split("=").str[0]
df['var2'] = df['var'].str.split("=").str[1]

In [60]:
df

,var,var1,var2
0,colsample_bytree=0.4,colsample_bytree,0.4
1,learning_rate=0.02,learning_rate,0.02
2,max_delta_step=0.8,max_delta_step,0.8
3,max_depth=None,max_depth,None
4,metric='binary_logloss',metric,'binary_logloss'
5,n_estimators=None,n_estimators,None
6,num_iterations=450,num_iterations,450
7,num_leaves=3,num_leaves,3
8,objective='binary',objective,'binary'
9,random_state=42,random_state,42


In [28]:
df['var'] = df['var'].str.split("=")
df

,var
0,"[colsample_bytree, 0.4]"
1,"[learning_rate, 0.02]"
2,"[max_delta_step, 0.8]"
3,"[ max_depth, None]"
4,"[metric, 'binary_logloss']"
5,"[n_estimators, None]"
6,"[ num_iterations, 450]"
7,"[num_leaves, 3]"
8,"[objective, 'binary']"
9,"[ random_state, 42]"


In [52]:
df['var1'] = df['var'].str[0]
df['var2'] = df['var'].str[1]

In [53]:
df

,var,var1,var2
0,"[colsample_bytree, 0.4]",colsample_bytree,0.4
1,"[learning_rate, 0.02]",learning_rate,0.02
2,"[max_delta_step, 0.8]",max_delta_step,0.8
3,"[ max_depth, None]",max_depth,None
4,"[metric, 'binary_logloss']",metric,'binary_logloss'
5,"[n_estimators, None]",n_estimators,None
6,"[ num_iterations, 450]",num_iterations,450
7,"[num_leaves, 3]",num_leaves,3
8,"[objective, 'binary']",objective,'binary'
9,"[ random_state, 42]",random_state,42


In [ ]:
df['new_column'] = df['codes'].str[0]


In [ ]:
df['new_column'] = [l[0] if len(l) > 0 else np.nan for l in df['codes']]

In [133]:
b = "colsample_bytree=0.4, learning_rate=0.02, max_delta_step=0.8, \
               max_depth=None, metric='binary_logloss', n_estimators=None, \
               num_iterations=450, num_leaves=3, objective='binary', \
               random_state=42, scale_pos_weight=2, subsample=None, \
'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.02, 'max_delta_step': 0.8, \
                'max_depth': None, 'metric': 'binary_logloss', 'n_estimators': None, \
                'num_iterations': 450, 'num_leaves': 3, 'objective': 'binary', \
                'scale_pos_weight': 2, 'subsample': None, \
Best Score: 0.7913043478260869, Best Index: 84963, \
train accuracy: 0.9826, val accuracy= 0.8621, test accuracy= 0.7568, \
precision : 0.6667, recall : 0.7143, f1score : 0.6897, roc : 0.7484, \
tn= 18, fp=5, fn=4, tp=10"


In [134]:
df = pd.DataFrame(b.split(", "), columns=['var'])

In [135]:
df['var1'] = df['var'].str.replace(":", "=")

In [136]:
df['var2'] = df['var1'].str.split("=")

In [140]:
df['param'] = df['var2'].str[0]
df['value'] = df['var2'].str[1]

In [143]:
df1 = df[['param', 'value']]

In [144]:
df1

,param,value
0,colsample_bytree,0.4
1,learning_rate,0.02
2,max_delta_step,0.8
3,max_depth,None
4,metric,'binary_logloss'
5,n_estimators,None
6,num_iterations,450
7,num_leaves,3
8,objective,'binary'
9,random_state,42
